In [2]:
import json
import pandas as pd
import pdal
import urllib
from urllib.request import urlopen
import shapely
from shapely.geometry import Polygon
import geopandas
import laspy
import rasterio
from rasterio.plot import show
from rasterio import mask
import folium
import fiona

In [4]:
x = geopandas.read_file('../geosome/shp/IA_FullState.shp')
x.plot(cmap='jet',column='NAME_4')

AttributeError: module 'fiona' has no attribute 'path'

In [3]:
from glob import glob
def get_shp_from_tif(tif_path:str, shp_file_path:str) -> None:
    raster = rasterio.open(tif_path)
    bounds = raster.bounds

    df = geopandas.GeoDataFrame({"id":1,"geometry":[box(*bounds)]})
   
    # save to file
    df.to_file(shp_file_path)
    print('Saved..')

In [4]:
from shapely.geometry import box
get_shp_from_tif('../geosome/tif/IA_FullState.tif', shp_file_path='../geosome/shp/combi.shp')

TypeError: 'NoneType' object is not callable

In [1]:
#import sys
#sys.path.append("../geosome/")
from geosome import geo_fetch

ModuleNotFoundError: No module named 'geosome'

In [3]:
load = load_file("([-10425171.940, -10423171.940], [5164494.710, 5166494.710])","IA_FullState/")
load.get_raster_terrain()

In [2]:
DATASET_PATH = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"
JSON_PATH = "../data/data_usgs.json"
        
def get_data(dataset_path:str,json_path:str):
    geo_dict = {'Region':[],
                'File_path':[],
                'Bounds':[],
                'WKT':[]
                }
    with open('../filename.txt') as filenames:
        for f in filenames:
            f = f.strip('\n')
            geo_dict['Region'].append(f)
            file = dataset_path + f +"ept.json"
            geo_dict['File_path'].append(file)
            try:
                meta = json.load(urlopen(file))
                wkt = meta["srs"]["wkt"]
                bounds = meta["bounds"]
            except:
                bounds = None                            
            geo_dict['Bounds'].append(bounds)    
            geo_dict['WKT'].append(wkt)
            
    with open(json_path,"w") as json_file:
        json.dump(geo_dict,json_file)                 

if __name__ == "__main__":
    get_data(DATASET_PATH,JSON_PATH)

A bound is an array of 6 numbers
[xmin, ymin,zmin,xmax,ymax,zmax]
We want to retain xmin,ymin,xmax,ymax

In [7]:
df = pd.read_json("../data/data_usgs.json")
df.head()

,Region,File_path,Bounds,WKT
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -12414, -17321558, 809116...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -19027, -15691854, 10976...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -45314, -16381190, 960963...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -35030, -14943073, 808364...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -166851, -16570284, 86397...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."


In [11]:
y=[]
for r in df['Region']:
    x = r.strip('/')
    y.append(x)
y

['AK_BrooksCamp_2012',
 'AK_Coastal_2009',
 'AK_Fairbanks-NSBorough_2010',
 'AK_Juneau_2012',
 'AK_Kenai_2008',
 'AK_MatanuskaSusitna-Lot1_2011',
 'AK_MatanuskaSusitna-Lot2_2011',
 'AK_NomeCreek_2010',
 'AK_NorthSlope_B10_2018',
 'AK_NorthSlope_B11_2018',
 'AK_NorthSlope_B12_2018',
 'AK_NorthSlope_B13_2018',
 'AK_NorthSlope_B14_2018',
 'AK_NorthSlope_B1_2018',
 'AK_NorthSlope_B2_2018',
 'AK_NorthSlope_B3_2018',
 'AK_NorthSlope_B4_2018',
 'AK_NorthSlope_B5_2018',
 'AK_NorthSlope_B6_2018',
 'AK_NorthSlope_B7_2018',
 'AK_NorthSlope_B8_2018',
 'AK_NorthSlope_B9_2018',
 'AK_ValdezB_2007',
 'AK_Valdez_2007',
 'AK_YukonFlats_2009',
 'AL_25Co_TL_2017',
 'AL_BaldwinCo-East_2011',
 'AL_BaldwinCo-West_2011',
 'AL_BlountCo_2010',
 'AL_ColbertLauderdale_2011',
 'AL_NorthAL_2019',
 'AL_TriCounty_2010',
 'AR-TN_NRCS_CaneCreek_2010',
 'AR-TN_NRCS_Sharkey_2010',
 'ARRA-AK_EkluntaGlacier_2010',
 'ARRA-AL_MobileBay_2010',
 'ARRA-AL_TennesseeRiver_2010',
 'ARRA-CA_CentralCoast-Z3_2010',
 'ARRA-CA_CentralC

In [9]:
df.head()

,Region,File_path,Bounds,WKT
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -12414, -17321558, 809116...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -19027, -15691854, 10976...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -45314, -16381190, 960963...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -35030, -14943073, 808364...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -166851, -16570284, 86397...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."


In [4]:
#Drop the z min and z max values
for i in df['Bounds']:
    del i[2]
for i in df['Bounds']:
    del i[4]

In [5]:
df.Bounds.loc[:5]

0      [-17347360, 8065364, -17321558, 8091166]
1    [-15730544, 10937407, -15691854, 10976097]
2      [-16471700, 9519129, -16381190, 9609639]
3      [-15014449, 8012267, -14943073, 8083643]
4      [-16906356, 8303726, -16570284, 8639798]
5      [-16782341, 8663224, -16678201, 8767364]
Name: Bounds, dtype: object

bounds_list = []
x = df.Bounds.loc[:5]
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half],a_list[half:]

for i in df['Bounds']:
    bounds = []
    a,b = split_list(i)
    bounds = tuple([a,b])
    bounds_list.append(bounds)

df['Bounds'] =bounds_list

In [6]:
df.sample(5)

,Region,File_path,Bounds
847,USGS_LPC_IL_District4_Warren_2014_LAS_2018/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-10121233, 4956438, -10053481, 5024190]"
332,KY_Eastern_B2_2019/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-9412640, 4438276, -9122740, 4728176]"
557,SC_DarlingtonCo_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-8937867, 4035407, -8865713, 4107561]"
679,USGS_LPC_AL_AGIO_B2_2016_LAS_2019/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-9890337, 3803583, -9727367, 3966553]"
13,AK_NorthSlope_B1_2018/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15990539, 11105651, -15983447, 11112743]"


In [26]:
#df.to_csv("../data/data.csv",index=False)

In [11]:
#df.to_json("../data/clean_data.json")

In [9]:
polygon = []
for b in df.Bounds:
    MINX,MINY,MAXX,MAXY = [b[0],b[1],b[2],b[3]]
    poly = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))
    polygon.append(poly)

In [10]:
df['geometry'] = polygon
df.head()

,Region,File_path,Bounds,geometry
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -17321558, 8091166]","POLYGON ((-17347360 8065364, -17347360 8091166..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -15691854, 10976097]","POLYGON ((-15730544 10937407, -15730544 109760..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -16381190, 9609639]","POLYGON ((-16471700 9519129, -16471700 9609639..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -14943073, 8083643]","POLYGON ((-15014449 8012267, -15014449 8083643..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -16570284, 8639798]","POLYGON ((-16906356 8303726, -16906356 8639798..."


In [ ]:
x,y = 

In [9]:
gdf = geopandas.GeoDataFrame(df)
gdf.head()

,Region,File_path,Bounds,geometry
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -17321558, 8091166]","POLYGON ((-17347360.000 8065364.000, -17347360..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -15691854, 10976097]","POLYGON ((-15730544.000 10937407.000, -1573054..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -16381190, 9609639]","POLYGON ((-16471700.000 9519129.000, -16471700..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -14943073, 8083643]","POLYGON ((-15014449.000 8012267.000, -15014449..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -16570284, 8639798]","POLYGON ((-16906356.000 8303726.000, -16906356..."


In [10]:
gdf['area'] = gdf.area
# Returns a GeoSeries of points for each geometric centroid.
gdf['centroid'] = gdf.centroid
# Returns a GeoSeries of geometries representing the point or smallest rectangular
# polygon (with sides parallel to the coordinate axes) that contains each object.
gdf['envelope'] = gdf.envelope

In [14]:
gdf = gdf.drop(['File_path','Bounds'],axis=1)
gdf.sample()

,Region,geometry,area,centroid,envelope
1544,VT_RockRiver_2008/,"POLYGON ((-8159821.000 5579646.000, -8159821.0...",1.999521e+09,POINT (-8137463.000 5602004.000),"POLYGON ((-8159821.000 5579646.000, -8115105.0..."


In [12]:
gdf.crs = "EPSG:4326"
print(gdf.crs)

EPSG:4326


In [13]:
gdf.sample()

,Region,File_path,Bounds,geometry,area,centroid,envelope
38,ARRA-CA_CentralCoast-Z4_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-13590521, 4270733, -13426673, 4434581]","POLYGON ((-13590521.000 4270733.000, -13590521...",2.684617e+10,POINT (-13508597.000 4352657.000),"POLYGON ((-13590521.000 4270733.000, -13426673..."


In [25]:
m = folium.Map(location=[37.0902,-95.7129], zoom_start=10, tiles='CartoDB positron')
m

In [28]:
gdf['lat'] = gdf.centroid.y
gdf['lon'] = gdf.centroid.x
gdf.head()

/tmp/ipykernel_19979/3024331562.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y
/tmp/ipykernel_19979/3024331562.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x


,Region,geometry,area,centroid,envelope,lat,lon
0,AK_BrooksCamp_2012/,"POLYGON ((-17347360.000 8065364.000, -17347360...",6.657432e+08,POINT (-17334459.000 8078265.000),"POLYGON ((-17347360.000 8065364.000, -17321558...",8078265.0,-17334459.0
1,AK_Coastal_2009/,"POLYGON ((-15730544.000 10937407.000, -1573054...",1.496916e+09,POINT (-15711199.000 10956752.000),"POLYGON ((-15730544.000 10937407.000, -1569185...",10956752.0,-15711199.0
2,AK_Fairbanks-NSBorough_2010/,"POLYGON ((-16471700.000 9519129.000, -16471700...",8.192060e+09,POINT (-16426445.000 9564384.000),"POLYGON ((-16471700.000 9519129.000, -16381190...",9564384.0,-16426445.0
3,AK_Juneau_2012/,"POLYGON ((-15014449.000 8012267.000, -15014449...",5.094533e+09,POINT (-14978761.000 8047955.000),"POLYGON ((-15014449.000 8012267.000, -14943073...",8047955.0,-14978761.0
4,AK_Kenai_2008/,"POLYGON ((-16906356.000 8303726.000, -16906356...",1.129444e+11,POINT (-16738320.000 8471762.000),"POLYGON ((-16906356.000 8303726.000, -16570284...",8471762.0,-16738320.0


In [27]:
# Estimate centroids of the country
# --------------------
cent_x =gdf['geometry'].centroid.x.mean()
cent_y =gdf['geometry'].centroid.y.mean()

# Create a map object using Folium
# --------------------
map_usa = folium.Map(location=[cent_y, cent_x], zoom_start=7, tiles='OpenStreetMap')  

# Create the choropleth map
# --------------------
choro = folium.Choropleth(geo_data=gdf,
                         name='choropleth',     
                          data=usa_g,
                          columns=['NAME_2', 'area'], 
#                           key_on='feature.NAME_2', 
                          fill_color='YlOrRd',
                          fill_opacity=0.6, 
                          line_opacity=0.8,
                          legend_name= "Area size across USA's subregion"
                          ).add_to(map_usa) 

# add labels to map
choro.geojson.add_child(folium.features.GeoJsonTooltip(fields=['NAME_2', 'area'],
                                                       aliases=['County', 'Mass'], 
                                                       labels = False))
folium.LayerControl().add_to(map_usa)


map_usa

/tmp/ipykernel_19979/3195566992.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent_x =gdf['geometry'].centroid.x.mean()
/tmp/ipykernel_19979/3195566992.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent_y =gdf['geometry'].centroid.y.mean()


NameError: name 'usa_g' is not defined